In [1]:
from operator import truediv
import cv2
import time
import torch
import numpy as np

import os
import sys
from tempfile import gettempdir

from torch.utils.tensorboard import SummaryWriter

from clearml import Task
sys.path.insert(0, 'py_scripts/') 

task = Task.init(project_name="bogdoll/Anomaly_detection_Moritz", task_name="QAgent_local", output_uri="https://tks-zx-01.fzi.de:8081")
task.set_base_docker(
            "nvcr.io/nvidia/pytorch:21.10-py3",
            docker_setup_bash_script="apt-get update && apt-get install -y python3-opencv",
            docker_arguments="-e NVIDIA_DRIVER_CAPABILITIES=all"  # --ipc=host",   
            )
# Remote Execution on FZI XZ
# task.set_base_docker(
#             "tks-zx-01.fzi.de/autonomous-agents/core-carla:21.10",
#             docker_setup_bash_script="apt-get update && apt-get install -y python3-opencv",
#             docker_arguments="-e NVIDIA_DRIVER_CAPABILITIES=all",  # --ipc=host",
#         )
# # PyTorch fix for version 1.10, see https://github.com/pytorch/pytorch/pull/69904
# task.add_requirements(
#     package_name="setuptools",
#     package_version="59.5.0",
# )
# task.add_requirements(
#     package_name="moviepy",
#     package_version="1.0.3",
# )
# task.execute_remotely('rtx3090', clone=False, exit_process=True) 

ClearML Task: created new task id=3a765deafb374c689426fe4f5be44634
ClearML results page: https://tks-zx.fzi.de:8080/projects/013741c08abc4e298d7346004c9c73dd/experiments/3a765deafb374c689426fe4f5be44634/output/log


In [2]:
                                                                        # http://tks-zx-01.fzi.de:8080/workers-and-queues/queues

from env_carla import Environment
from env_carla import IM_HEIGHT
from env_carla import IM_WIDTH


from training import Training

from training import N_EPISODES
from training import TARGET_UPDATE
from training import EPS_START

# The learned Q value rates (state,action) pairs
# A CNN with a state input can rate possible actions, just as a classifier would

PREVIEW = False
VIDEO_EVERY = 1_000


def main():
    writer = SummaryWriter()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     env = Environment(host="tks-fly.fzi.de", port=2000)
    env = Environment(host="localhost", port=2000, random_spawn=False)
    env.init_ego()

    trainer = Training(writer, device)

    epsilon = EPS_START
    reward_best = -1000

    for i in range(N_EPISODES):

        reward_per_episode = 0
        start = time.time()
        n_frame = 1

        obs_current = env.reset()
        env.spawn_anomaly()
        obs_current = np.transpose(obs_current[0], (2,1,0))
        obs_current = np.array([obs_current])
#         print(obs_current.shape)
        obs_current = torch.as_tensor(obs_current)

        chw_list = []

        while True:
            if PREVIEW:
                cv2.imshow("", env.observation)
                cv2.waitKey(1)

#             chw = obs_current.squeeze(0)  # Remove batch information from BCHW
            chw_list.append(obs_current)

            # Perform action on observation and buildup replay memory
            if i % VIDEO_EVERY == 0:
                action = trainer.select_action(obs_current, 0)
            else:
                action = trainer.select_action(obs_current, epsilon)
            obs_next, reward, done, _ = env.step(action)
            reward_per_episode += reward
            reward_torch = torch.tensor([reward], device=device)  # For compatibility with PyTorch

            if (
                time.time() - start
            ) > 30:  # Since the agent can simply stand now, the episode should terminate after 30 seconds
                done = True

            if done:
                obs_next = None
            else:
                obs_next = obs_next[0] #no segemntation
                obs_next = np.transpose(obs_next, (2,1,0))
                obs_next = np.array([obs_next])
                obs_next = torch.as_tensor(obs_next)
            
            # Python tuples () https://www.w3schools.com/python/python_tuples.asp
            trainer.replay_memory.push(obs_current, action, obs_next, reward_torch, done)
            
            obs_current = obs_next

            # Optimization on policy model (I believe this could run in parallel to the data collection task)
            trainer.optimize(i)

            if done:
                end = time.time()
                duration = end - start
                writer.add_scalar("Reward per episode", reward_per_episode, i)
                writer.add_scalar("Duration before crash/seconds", duration, i)
                writer.add_scalar("Frames before crash/frames", n_frame, i)

                if reward_per_episode > reward_best:
                    reward_best = reward_per_episode
                    tchw_list = torch.stack(chw_list)  # Adds "list" like entry --> TCHW
                    tchw_list = torch.squeeze(tchw_list)
                    name = "DQN Champ: " + str(reward_per_episode)
                    writer.add_video(
                        tag=name, vid_tensor=tchw_list.unsqueeze(0), global_step=i
                    )  # Unsqueeze adds batch --> BTCHW
                    torch.save(trainer.policy_net.state_dict(), os.path.join(gettempdir(), "dqn_" + str(i) + ".pt"))
                break

            n_frame += 1

        # Save video of episode to ClearML https://github.com/pytorch/pytorch/issues/33226
        if i % VIDEO_EVERY == 0:
            tchw_list = torch.stack(chw_list)  # Adds "list" like entry --> TCHW
            tchw_list = torch.squeeze(tchw_list)
            writer.add_video(
                tag="DQN Agent", vid_tensor=tchw_list.unsqueeze(0), global_step=i
            )  # Unsqueeze adds batch --> BTCHW

        # Update the target network, copying all weights and biases in DQN
        if i % TARGET_UPDATE == 0:
            trainer.target_net.load_state_dict(trainer.policy_net.state_dict())

        # Decay epsilon
        writer.add_scalar("Exploration-Exploitation/epsilon", epsilon, i)
        epsilon = trainer.decay_epsilon(epsilon)

    writer.flush()



======> WARNING! Git diff to large to store (598kb), skipping uncommitted changes <======


In [3]:
main()

46049.39923641465 iterations until EPS_END is reached
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32

nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
n

nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
n

nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
n

nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
n

...

nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([30, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
n

nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
n

nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([30, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([31, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
nfns:  torch.Size([32, 3, 256, 256])
n

RuntimeError: Spawn failed because of collision at spawn position

2022-09-24 12:52:54,286 - clearml.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###
